In [1]:
import numpy as np
import joblib
import pandas as pd

def test_mi_prediction_model():
    print("Testing MI Prediction Model")
    print("--------------------------")

    try:
        print("Loading model from '/content/mi_best_model.pkl'...")
        model = joblib.load('/content/mi_best_model.pkl')
        print("Model loaded successfully!")
    except FileNotFoundError:
        print("Error: Model file '/content/mi_best_model.pkl' not found.")
        return

    # Creating a test sample
    # ca_0.0 (0 = no, 1 = yes), thal_7.0 (0 = no, 1 = yes), cp_4.0 (0 = no, 1 = yes),
    # asymptomatic_chest_pain (0 = no, 1 = yes), age_sex (combined feature),
    # combined_risk_score, oldpeak (ST depression), thalach (max heart rate),
    # thalach_exang (interaction), thal_3.0 (0 = no, 1 = yes)
    sample = np.array([
        [1.0, 0.0, 0.0, 0.0, 45.0, 2.5, 1.2, 150.0, 0.0, 1.0]
    ])

    feature_names = [
        'ca_0.0', 'thal_7.0', 'cp_4.0', 'asymptomatic_chest_pain',
        'age_sex', 'combined_risk_score', 'oldpeak', 'thalach',
        'thalach_exang', 'thal_3.0'
    ]

    print("\nSample patient data:")
    for feature, value in zip(feature_names, sample[0]):
        print(f"  {feature}: {value}")

    try:
        prediction = model.predict(sample)[0]
        probability = model.predict_proba(sample)[0, 1]

        print("\nPrediction results:")
        print(f"  MI Risk: {'Yes' if prediction == 1 else 'No'}")
        print(f"  Probability of MI: {probability:.2%}")

        if hasattr(model, 'coef_'):
            print("\nFeature contributions to this prediction:")
            coefficients = model.coef_[0]
            contrib = coefficients * sample[0]

            sorted_indices = np.argsort(np.abs(contrib))[::-1]

            for idx in sorted_indices:
                feature = feature_names[idx]
                contribution = contrib[idx]
                print(f"  {feature}: {contribution:.4f} ({'increases' if contribution > 0 else 'decreases'} risk)")

    except Exception as e:
        print(f"Error during prediction: {e}")

    print("\nWould you like to test another patient? (y/n)")
    response = input()

    if response.lower() == 'y':
        print("\nEnter values for the patient:")
        new_sample = []

        for feature in feature_names:
            value = float(input(f"Enter value for {feature}: "))
            new_sample.append(value)

        new_sample = np.array([new_sample])

        prediction = model.predict(new_sample)[0]
        probability = model.predict_proba(new_sample)[0, 1]

        print("\nPrediction results:")
        print(f"  MI Risk: {'Yes' if prediction == 1 else 'No'}")
        print(f"  Probability of MI: {probability:.2%}")

if __name__ == "__main__":
    test_mi_prediction_model()1

Testing MI Prediction Model
--------------------------
Loading model from '/content/mi_best_model.pkl'...
Model loaded successfully!

Sample patient data:
  ca_0.0: 1.0
  thal_7.0: 0.0
  cp_4.0: 0.0
  asymptomatic_chest_pain: 0.0
  age_sex: 45.0
  combined_risk_score: 2.5
  oldpeak: 1.2
  thalach: 150.0
  thalach_exang: 0.0
  thal_3.0: 1.0

Prediction results:
  MI Risk: No
  Probability of MI: 0.00%

Feature contributions to this prediction:
  thalach: -278.1794 (decreases risk)
  age_sex: 18.2881 (increases risk)
  combined_risk_score: 2.0422 (increases risk)
  thal_3.0: 1.4885 (increases risk)
  oldpeak: 0.9802 (increases risk)
  ca_0.0: -0.2813 (decreases risk)
  thalach_exang: -0.0000 (decreases risk)
  asymptomatic_chest_pain: 0.0000 (decreases risk)
  cp_4.0: -0.0000 (decreases risk)
  thal_7.0: 0.0000 (decreases risk)

Would you like to test another patient? (y/n)
y

Enter values for the patient:
Enter value for ca_0.0: 1
Enter value for thal_7.0: 1
Enter value for cp_4.0: 1
En